# Retrieval Augmented Generation

## Setup API clients

In [22]:
import os

# import azure.identity
import dotenv
import openai
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery
from azure.core.credentials import AzureKeyCredential
import json

dotenv.load_dotenv()

# azure_credential = azure.identity.DefaultAzureCredential()

# Set up OpenAI client based on environment variables
dotenv.load_dotenv()
AZURE_OPENAI_SERVICE = "ragsearchpocopenai"
AZURE_OPENAI_ADA_DEPLOYMENT = "text-embedding-ada-002"

# azure_credential = azure.identity.DefaultAzureCredential()
azure_openai_key = os.getenv("AZURE_OPENAI_KEY")
# token_provider = azure.identity.get_bearer_token_provider(azure_openai_key,
#     "https://cognitiveservices.azure.com/.default")
openai_client = openai.AzureOpenAI(
    api_version="2024-06-01",
    azure_endpoint=f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com",
    api_key=azure_openai_key,
)


def get_embedding(text):
    get_embeddings_response = openai_client.embeddings.create(
        model=AZURE_OPENAI_ADA_DEPLOYMENT, input=text
    )
    return get_embeddings_response.data[0].embedding


# Initialize Azure search client
AZURE_SEARCH_SERVICE = "ragsearchpocsch"
AZURE_SEARCH_ENDPOINT = f"https://{AZURE_SEARCH_SERVICE}.search.windows.net"

AZURE_SEARCH_FULL_INDEX = "ragsearch"
search_client = SearchClient(
    AZURE_SEARCH_ENDPOINT,
    AZURE_SEARCH_FULL_INDEX,
    credential=AzureKeyCredential(os.getenv("AZURE_SEARCH_KEY")),
)

## Prepare user question

In [23]:
user_question = "What does a product manager do?"
user_question_vector = get_embedding(user_question)

In [16]:
user_question_vector

[2.5574694518581964e-05,
 -0.019146429374814034,
 -0.008829147554934025,
 -0.002112617716193199,
 -0.021192612126469612,
 0.019691191613674164,
 -0.01634289138019085,
 0.019863922148942947,
 -0.014575733803212643,
 -0.027424169704318047,
 0.002157461130991578,
 -0.00023335125297307968,
 -0.011001556180417538,
 0.0025095639284700155,
 0.008722851984202862,
 -0.013778519816696644,
 0.0196779053658247,
 -0.0006045541376806796,
 -0.009241041727364063,
 -0.015718407928943634,
 -0.01570511981844902,
 -0.01743241772055626,
 0.002522850874811411,
 0.0014358160551637411,
 -0.004490973427891731,
 0.006331209558993578,
 0.03327040746808052,
 -0.002659041667357087,
 0.005045701749622822,
 -0.0039628189988434315,
 0.01686108112335205,
 -0.002735441317781806,
 -0.004481008276343346,
 0.0067929294891655445,
 0.00746724009513855,
 0.014389717020094395,
 0.011273938231170177,
 0.004235200583934784,
 0.0005551434587687254,
 -0.0030809007585048676,
 0.014376429840922356,
 0.008822504431009293,
 0.0193058

## Retrieve matching documents

The search call below does a **hybrid search**, performing both a full-text search and a vector search in parallel.
It merges those results using Reciprocal Rank Fusion (RRF). 
Finally, it re-ranks the merged results using the AI Search semantic ranker, a re-ranking model.

In [5]:
r = search_client.search(
        user_question,
        top=5, 
        vector_queries=[
                VectorizedQuery(vector=user_question_vector, k_nearest_neighbors=50, fields="vector")],
        query_type="semantic",
        semantic_configuration_name="ragsearch-semantic-config")

sources = "\n\n".join([f"[{doc['title']}]: {doc['chunk']}\n" for doc in r])

print(sources)

[role_library.pdf]: -->

· Knowledge of industry trends, technologies, and regulations.

· Excellent communication and presentation skills.


#### Manager of Product Management

Job Title: Manager of Product Management, Contoso Electronics

Job Summary:

The Manager of Product Management is responsible for overseeing the product
management team, driving product development and marketing strategy for Contoso
Electronics. This individual will be accountable for the successful launch of new products
and the implementation of product life-cycle management processes. The Manager of
Product Management will collaborate with internal teams, such as engineering, sales,
marketing, and finance, as well as external partners, suppliers, and customers to ensure
successful product execution.

Responsibilities:

· Lead the product management team and provide guidance on product strategy, design,
development, and launch.

· Develop and implement product life-cycle management processes.

· Monitor and a

## Send question and documents to the model

In [24]:
SYSTEM_MESSAGE = """
Assistant helps company employees questions about the employee handbook. Be brief in your answers.
Answer ONLY with the facts listed in the list of sources below.
If there isn't enough information below, say you don't know. Do not generate answers that don't use the sources below.
Each source has a name followed by colon and the actual information, include the source name for each fact you use.
Use square brackets to reference the source, for example [info1.txt].
"""
USER_MESSAGE = user_question + "\nSources: " + sources

# Now we can use the matches to generate a response
response = openai_client.chat.completions.create(
    model="gpt-4o-mini",
    temperature=0.7,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

answer = response.choices[0].message.content
print(answer)

A product manager oversees product development and marketing strategy, collaborates with internal teams, analyzes industry trends, develops product roadmaps, and ensures successful product launches. They manage the product lifecycle, including strategy, design, development, and execution. Qualifications typically include a relevant bachelor's degree and several years of experience in product management [role_library.pdf].
